In [16]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import os
import numpy as np
import json

In [17]:


def produce_brut(assureur):
    with open('data/ocr/{}.json'.format(assureur), 'r',encoding="utf8") as f:
        data = json.load(f)
    pages_content=data["responses"]
    num_page=0
    df=[]
    for page in pages_content:
        num_page+=1
        if "fullTextAnnotation" not in page:
            continue
        p=page["fullTextAnnotation"]["pages"]
        for e in p:
            blocks=e["blocks"]
            page_features=[]
            for block in blocks:
                for para in block["paragraphs"]:
                    # collect text
                    text = ""
                    for word in para["words"]:
                        #print("-----")
                        #print(word)
                        for symbol in word["symbols"]:
                            if symbol["confidence"]>=0.8:
                                text += symbol["text"]
                        text+=" "
                    # extract bounding box features
                    x_list = []
                    y_list = []
                    for v in para["boundingBox"]["normalizedVertices"]:
                        x_list.append(v["x"])
                        y_list.append(v["y"])
                    f = {}
                    f["num_page"]=num_page
                    f["text"] = text
                    f["width"] = max(x_list) - min(x_list)
                    f["height"] = max(y_list) - min(y_list)
                    f["area"] = f["width"] * f["height"]
                    f["chars"] = len(text)
                    f["char_size"] = f["area"] / f["chars"] if f["chars"] > 0 else 0
                    f["pos_x"] = (f["width"] / 2.0) + min(x_list)
                    f["pos_y"] = (f["height"] / 2.0) + min(y_list)
                    f["aspect"] = f["width"] / f["height"] if f["height"] > 0 else 0
                    f["layout"] = "h" if f["aspect"] > 1 else "v"
                    f["x0"]=x_list[0]
                    f["x1"]=x_list[1]
                    f["y0"]=y_list[0]
                    f["y1"]=y_list[1]
                    page_features.append(f)
            df=df+page_features
    df=pd.DataFrame(df)
    df["assureur"]=assureur
    return df

In [18]:
produce_brut("allianz-1-to-94")

,num_page,text,width,height,area,chars,char_size,pos_x,pos_y,aspect,layout,x0,x1,y0,y1,assureur
0,1,Allianz,0.250420,0.030879,0.007733,9,0.000859,0.180672,0.112827,8.109758,h,0.055462,0.305882,0.097387,0.098575,allianz-1-to-94
1,1,ALLIANZ VIE,0.117647,0.010689,0.001258,12,0.000105,0.114286,0.215558,11.006553,h,0.055462,0.173109,0.210214,0.210214,allianz-1-to-94
2,1,Rapport sur la solvabilité,0.663866,0.041568,0.027595,27,0.001022,0.392437,0.271378,15.970699,h,0.060504,0.724370,0.256532,0.250594,allianz-1-to-94
3,1,Exercice 2022,0.121008,0.010689,0.001293,14,0.000092,0.115966,0.437648,11.320995,h,0.055462,0.176471,0.432304,0.432304,allianz-1-to-94
4,1,et la situation financière,0.638655,0.033254,0.021238,27,0.000787,0.613445,0.321853,19.205268,h,0.294118,0.932773,0.306413,0.305226,allianz-1-to-94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4166,94,Entreprise régie par le Code des assurances So...,0.322689,0.038005,0.012264,107,0.000115,0.243697,0.874109,8.490757,h,0.082353,0.405042,0.856294,0.855107,allianz-1-to-94
4167,94,1 cours Michelet - CS 30051 92076 Paris La Déf...,0.196639,0.022565,0.004437,57,0.000078,0.180672,0.906770,8.714202,h,0.082353,0.278992,0.895487,0.895487,allianz-1-to-94
4168,94,340 234 962 R.C.S. Nanterre,0.183193,0.007126,0.001305,28,0.000047,0.173950,0.927553,25.708092,h,0.082353,0.265546,0.923991,0.923991,allianz-1-to-94
4169,94,,0.023529,0.014252,0.000335,1,0.000335,0.850420,0.923991,1.650981,h,0.838655,0.862185,0.916865,0.916865,allianz-1-to-94


In [35]:
type(produce_brut("allianz-1-to-94"))

pandas.core.frame.DataFrame

In [44]:
import pandas as pd

# Supposons que votre DataFrame s'appelle 'dataframe'
file_path = 'dataframe.csv'  # Chemin de sauvegarde temporaire
produce_brut("allianz-1-to-94").to_csv(file_path, index=False, encoding='utf-8')

print(f"Le DataFrame a été sauvegardé sous le nom 'dataframe.csv'. Vous pouvez maintenant le télécharger.")


Le DataFrame a été sauvegardé sous le nom 'dataframe.csv'. Vous pouvez maintenant le télécharger.


In [45]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [108]:
dataframe=produce_brut("allianz-1-to-94")

In [109]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# 1. Sélection des colonnes pertinentes pour le clustering
features = dataframe[['width', 'height', 'area', 'chars', 'char_size', 'aspect', 'pos_x', 'pos_y']]

In [110]:
# 2. Standardisation des données (important pour K-Means)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [111]:
# 3. Application de K-Means avec 3 clusters
kmeans = KMeans(n_clusters=3, random_state=42)
dataframe['cluster'] = kmeans.fit_predict(features_scaled)


In [127]:
def nawfal(dataframe):
  for i in range (len(dataframe)-1):
    if len(dataframe["text"][i].split())>40:
      dataframe.loc[i, "cluster"] =1
    if len(dataframe["text"][i].split())<20 and len(dataframe["text"][i].split())>5:
      dataframe.loc[i, "cluster"] =0
    if len(dataframe["text"][i].split())<10 and dataframe["cluster"][i]==1:
      dataframe.loc[i, "cluster"] =0
    if len(dataframe["text"][i])<8:
      dataframe.loc[i, "cluster"] =2
  return dataframe
def nawfal2(dataframe):
  for i in range (len(dataframe)-1):
    if dataframe["cluster"][i]==0 : dataframe.loc[i, "cluster"] ='Titre'
    if dataframe["cluster"][i]==1 : dataframe.loc[i, "cluster"] ="Paragraphe"
    if dataframe["cluster"][i]==2 : dataframe.loc[i, "cluster"] ="inutile"
  return dataframe


In [61]:
numeric_columns = dataframe.select_dtypes(include=['number']).columns
print(dataframe.groupby('cluster')[numeric_columns].mean())

          num_page     width    height      area       chars  char_size  \
cluster                                                                   
0        53.161747  0.232218  0.010611  0.002661   41.807503   0.000059   
1        41.390034  0.771426  0.040513  0.029174  296.644330   0.000342   
2        56.079470  0.053013  0.010844  0.000619   12.624554   0.000039   

            pos_x     pos_y     aspect        x0        x1        y0  \
cluster                                                                
0        0.248897  0.472755  23.685437  0.132808  0.364991  0.467608   
1        0.475034  0.470516  24.287956  0.089362  0.860709  0.451045   
2        0.655996  0.393776   6.709342  0.629520  0.682172  0.390798   

               y1  cluster  
cluster                     
0        0.467646      0.0  
1        0.450578      1.0  
2        0.388477      2.0  


In [123]:
dataframe=nawfal(dataframe)

In [128]:
dataframe=nawfal2(dataframe)

<ipython-input-127-aa00f61b0510>:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Titre' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  if dataframe["cluster"][i]==0 : dataframe.loc[i, "cluster"] ='Titre'


In [130]:
output_path = 'clustered_dataframe.xlsx'
dataframe.to_excel(output_path, index=False)
print(f"Le fichier avec les clusters a été sauvegardé ici : {output_path}")

Le fichier avec les clusters a été sauvegardé ici : clustered_dataframe.xlsx
